<a href="https://colab.research.google.com/github/jlopetegui98/Literary-Fine-Tuning-of-LLM/blob/main/FineTuning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine tuning Mistral-7B

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
!nvidia-smi

Mon Jan 22 21:48:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
import json

In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
dataset_name = "wilde_fine-tune_instructions_512_chat_format"

In [ ]:
# torch.set_default_device('cuda')

In [6]:
dir_data = f'./drive/MyDrive/DL-ENS/dataset/{dataset_name}.json'

In [7]:
data_dict = json.load(open(dir_data))
dataset = Dataset.from_dict(data_dict)
dataset = dataset.train_test_split(test_size = 0.1, seed = 42)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1396
    })
    test: Dataset({
        features: ['text'],
        num_rows: 156
    })
})

In [8]:
dataset['train']['text'][0]

'<s> [INST] This are the first lines of a work of fiction. Continue it., to listen once to Sancho Panza is both salutary and refreshing. As for his Sententiae, they differ very widely in character and subject. Some of them are ethical, such as \'Humility may be carried too far\'; some literary, as \'For one Froude there are a thousand Mrs. Markhams\'; and some scientific, as \'Objects which are near display more detail than those which are further off.\' Some, again, breathe a fine spirit of optimism, as \'Picturesqueness is the birthright of the bargee\'; others are jubilant, as \'Paint firm and be jolly\'; and many are purely autobiographical, such as No. 97, [/INST] \'Few of us understand what it is that we mean by Art.\' Nor is Mr. Quilter\'s manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, \'Similes drawn from eating and its results occur most readily to the mind.\' So he announces that \

In [9]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [10]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [11]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer):

In [12]:
# Monitering the LLM
wandb.login(key = "a182fc874550009f27ad8c011b668d3a04c55608")
run = wandb.init(project='Fine tuning mistral 7B', job_type="training", anonymous="allow")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: javier-lopetegui (nlp-tasks). Use `wandb login --relogin` to force relogin


In [13]:
#Hyperparamters for fine tuning
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 2,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    report_to="wandb"
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/1396 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,2.745800
60,2.542500
90,2.471000
120,2.441500
150,2.409200
180,2.366900
210,2.084100
240,2.068700
270,2.120700
300,2.065700


TrainOutput(global_step=348, training_loss=2.2918696896783235, metrics={'train_runtime': 18487.8463, 'train_samples_per_second': 0.151, 'train_steps_per_second': 0.019, 'total_flos': 6.101084433442406e+16, 'train_loss': 2.2918696896783235, 'epoch': 1.99})

In [15]:
trainer.evaluate()

{'eval_loss': 2.3142619132995605,
 'eval_runtime': 317.9268,
 'eval_samples_per_second': 0.491,
 'eval_steps_per_second': 0.063,
 'epoch': 1.99}

In [16]:
model_save_name = 'Mistral7B_fine_tuned_OscarWilde.pt'
path = f"./drive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

In [17]:
dataset['test']['text'][0]

"<s> [INST] This are the first lines of a work of fiction. Continue it.George Eliot, whose poetry is too abstract, and lacks all rhythmical life; Mrs. Carlyle, who wrote much better poetry than her husband, though this is hardly high praise; and Mrs. Browning, the first really great poetess in our literature. Nor are contemporary writers forgotten. Christina Rossetti, some of whose poems are quite priceless in their beauty; Mrs. Augusta Webster, Mrs. Hamilton King, Miss Mary Robinson, Mrs. Craik; Jean Ingelow, whose sonnet on An Ancient Chess King is like an exquisitely carved gem; Mrs. Pfeiffer; Miss May Probyn, a poetess with the true lyrical impulse of song, whose work is as [/INST] delicate as it is delightful; Mrs. Nesbit, a very pure and perfect artist; Miss Rosa Mulholland, Miss Katharine Tynan, Lady Charlotte Elliot, and many other well-known writers, are duly and adequately represented. On the whole, Mrs. Sharp's collection is very pleasant reading indeed, and the extracts giv

In [ ]:
prompt = "<s>This are the first lines of a work of fiction. Continue it.[INST]George Eliot, whose poetry is too abstract, and lacks all rhythmical life; Mrs. Carlyle, who wrote much better poetry than her husband, though this is hardly high praise; and Mrs. Browning, the first really great poetess in our literature.  Nor are contemporary writers forgotten.  Christina Rossetti, some of whose poems are quite priceless in their beauty; Mrs. Augusta Webster, Mrs. Hamilton King, Miss Mary Robinson, Mrs. Craik; Jean Ingelow, whose sonnet on An Ancient Chess King is like an exquisitely carved gem; Mrs. Pfeiffer; Miss May Probyn, a poetess with the true lyrical impulse of song, whose work is as delicate as it is delightful; Mrs. Nesbit, a very pure and perfect artist; Miss Rosa Mulholland, Miss Katharine Tynan, Lady Charlotte Elliot, and many other well-known writers, are duly and adequately represented.  On the whole, Mrs. Sharp's collection is very pleasant reading indeed, and the extracts given from the works of living poetesses are extremely remarkable, not merely for their absolute artistic excellence, but also for the light they throw upon the spirit of modern culture.  It is not, however, by any means a complete anthology.  Dame Juliana Berners is possibly too antiquated in style to be suitable to a modern audience.  But where is Anne Askew, who wrote a ballad in Newgate; and where is Queen Elizabeth, whose 'most sweet and sententious ditty' on Mary Stuart is so highly praised by Puttenham as an example of 'Exargasia,' or The Gorgeous in Literature?  Why is the Countess of Pembroke excluded?  Sidney's sister should surely have a place in any anthology of English verse.  Where is Sidney's niece, Lady Mary Wroth, to whom Ben Jonson dedicated The Alchemist?  Where is 'the noble ladie Diana Primrose,' who wrote A Chain of Pearl, or a memorial of the peerless graces and heroic virtues of Queen Elizabeth, of glorious memory?  Where is Mary Morpeth, the friend and admirer of Drummond of Hawthornden?  Where is the Princess Elizabeth, daughter of[/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=550)
result = pipe(prompt)
len(result[0]['generated_text'].split()), len(prompt.split())

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

(348, 336)

In [ ]:
n_model = trainer.model

In [ ]:
# prompt = "<s>This are the first lines of a work of fiction. Continue it.[INST]George Eliot, whose poetry is too abstract, and lacks all rhythmical life; Mrs. Carlyle, who wrote much better poetry than her husband, though this is hardly high praise[/INST]"
# pipe = pipeline(task="text-generation", model=n_model, tokenizer=tokenizer, max_length=550)
# result = pipe(prompt)
# len(result[0]['generated_text'].split()), len(prompt.split())